In [114]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import load_breast_cancer

# SVM com minimize scipy.optimize

In [102]:
class SVML:

    def __init__(self, C):
        self.C = C
        self.alpha = None
        self.w = None
        self.b = None

    def svm_fit(self, X, y, C=1.0):
        n_samples, n_features = X.shape
        K = np.dot(X, X.T)
        P = np.outer(y, y)*K
        constraints = [{'type': 'eq', 'fun': lambda alpha: np.dot(alpha, y)}]
        def objective(alpha):
            return 0.5 * np.dot(alpha, np.dot(P, alpha)) - np.sum(alpha)
        result = minimize(objective, np.zeros(n_samples), bounds=[(0, self.C) for _ in range(n_samples)], constraints=constraints)
        alpha = result.x
        return alpha

    def wb(self, X, y, alpha):

        w = np.dot(alpha*y, X)
        sv_idx = np.where((alpha > 0) & (alpha < self.C))[0]
        b = np.mean(y[sv_idx] - np.dot(X[sv_idx], w))

        return w, b



accuracy = lambda y_pred, y_true: (y_pred == y_true).sum()/(y_pred == y_true).shape[0]

In [103]:
data = load_breast_cancer()
X, y = data.data, data.target
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y)
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [105]:
svm = SVML(C=1.0)
alphas= svm.svm_fit(X_train, y_train)
w, b = svm.wb(X_train, y_train, alphas)
y_pred = np.sign(X_test.dot(w) + b)
score = accuracy(y_test, y_pred)
print(score)

0.9707602339181286


In [106]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.97      0.95      0.96        64
           1       0.97      0.98      0.98       107

    accuracy                           0.97       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171



# **Kernel**

In [107]:
def rbf(X1, X2, gamma=None):
    if gamma is None:
        gamma = 2 / X1.shape[-1]

    if np.ndim(X1) == 1 and np.ndim(X2) == 1:
        result = np.exp(-gamma * np.linalg.norm(X1 - X2)**2)
    elif (np.ndim(X1) > 1 and np.ndim(X2) == 1) or (np.ndim(X1) == 1 and np.ndim(X2) > 1):
        result = np.exp(-gamma * np.linalg.norm(X1 - X2, axis=1)**2)
    elif np.ndim(X1) > 1 and np.ndim(X2) > 1:
        result = np.exp(-gamma * np.linalg.norm(X1[:, np.newaxis] - X2[np.newaxis, :], axis=2)**2)
    return result

SVM GRADIENTE

In [115]:
def f_objective(alphas, p):
    f = np.sum(alphas) - 1/2*( np.sum(p * np.outer(alphas, alphas)))
    return f

def svm_fit(X, y, kernel, C=1.0, lr=1e-6, n_iters=300):
    n = X.shape[0]
    alphas = np.random.random(n)
    b = 0
    K = kernel(X, X)
    p = np.outer(y, y)*K

    for _ in range(n_iters):
        loss = f_objective(alphas, p)
        grad = np.ones(n) - p.dot(alphas)
        alphas += lr*grad

        alphas[alphas < 0] = 0
        alphas[alphas > C] = C

    id = np.where((alphas) > 0 & (alphas < C))[0]

    b = np.mean(y[id] - (alphas*y).dot(kernel(X, X[id])))

    return alphas, b

In [124]:
def svm_predict(X, X_train, y_train, alphas, b, kernel):
    decision = (alphas*y_train).dot(kernel(X_train, X)) + b

    return np.sign(decision)

alphas, b = svm_fit(X_train, y_train, rbf, C=1.0, lr=1e-3, n_iters=200)
y_p = svm_predict(X_test, X_train, y_train, alphas, b, rbf)
y_test = y_test.astype(float)
np.mean(y_test == y_p)

0.9649122807017544

In [125]:
print(classification_report(y_test, y_p))

              precision    recall  f1-score   support

        -1.0       0.97      0.94      0.95        64
         1.0       0.96      0.98      0.97       107

    accuracy                           0.96       171
   macro avg       0.97      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171

